------------------------------------------------------------------------
This notebook is a Multi Layer Perceptron(MLP) implementation for a Tabular data classification problem using Pytorch . Are **Neural Nets an overkill or do they lack performance on Tabular data** ? 

Well recently there has been a lot of buzz around utility of NN's in tabular data.In a study , Gradient boosting methods still outperform most scenarios of tabular data world . Related reading in this paper - [Deep Learning is not all you need](https://arxiv.org/pdf/2106.03253.pdf)

Please remember to **upvote this notebook** for encouragement , i'll be grateful :-)

--------------------------------------------------------------------------

### Background on the Dataset 📝
**What is Phishing ?**

It is a form of fraud in which the attacker tries to learn sensitive information such as login credentials or account information by pretending as a reputable entity or person via email or other communication channel means

Phishing is popular among attackers, since it is easier to trick someone into clicking a malicious link which seems legitimate

The **URL** of phishing websites may be **very similar to real websites** to the human eye, but they are different in IP. 

#### URL Basics

![URL Structure](https://i.imgur.com/elrD4Vl.png)

* **Domain name** portion is constrained since it has to be registered with a domain name Registrar
* **Subdomain name** and **Path** are fully controllable by the phisher

### Phishing dataset Exploration

In [ ]:

import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import f1_score as f1
from sklearn.metrics import confusion_matrix
from sklearn.metrics import *

#-- Pytorch specific libraries import -----#
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
df_data = pd.read_csv("../input/web-page-phishing-detection-dataset/dataset_phishing.csv")
df_data.shape

In [ ]:
df_data.head(5)

#### Quick Observations
* **URL** itself is present as a field
* Target Variable is : **status** with 
    * legitimate (considered as Label 1)
    * phishing   (considered as Label 0)
* **87 Features** around URL , **all Numerical** ,are from three different classes
    * 56 extracted from the Structure and Syntax of URLs (fields starting with : *nb_* , *shortest/longest* , *ratio/length* etc)
    * 24 extracted from the Content of their correspondent pages 
    * 7 are extracted by querying External Services

In [ ]:
#Encoding 'status' as label 1 & 0 , naming the field as target
df_data['target'] = pd.get_dummies(df_data['status'])['legitimate'].astype('int')
df_data.drop('status',axis = 1, inplace=True)
df_data[['url','target']].head(5)

**No Missing values in the dataset**

In [ ]:
tmp = df_data.isnull().sum().reset_index(name='missing_val')
tmp[tmp['missing_val']!= 0]

**Idenitfying Categorical columns** : If column has unique values lower than 0.002% of total records then categorizing it as Categorical

In [ ]:
likely_cat = {}
for var in df_data.iloc[:,1:].columns:
    likely_cat[var] = 1.*df_data[var].nunique()/df_data[var].count() < 0.002 

num_cols = []
cat_cols = []
for col in likely_cat.keys():
    if (likely_cat[col] == False):
        num_cols.append(col)
    else:
        cat_cols.append(col)


#### Correlation Plot for the Numerical(continuous) features


In [ ]:
#Taking all columns except URL 
corr = df_data[num_cols].corr()

fig = plt.figure(figsize=(12,12),dpi=80)
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, cmap='BuPu', robust=True, center=0,
            square=True, linewidths=.5)
plt.title('Correlation of Numerical(Continous) Features', fontsize=15,font="Serif")
plt.show()

**Observations** :Few highly correlated features as one would expect.Nothing jumps out extraordinary here.

*length_words_raw* is expected to be highly correlated with *length_url* . Similar behaviour for other correlated feature pairs


#### Distribution of Mean values of the Numerical features across Target variable

In [ ]:
df_distr =df_data.groupby('target')[num_cols].mean().reset_index().T
df_distr.rename(columns={0:'0_Label',1:"1_Label"}, inplace=True)

#plt.style.use('ggplot')
plt.rcParams['axes.facecolor']='w'
ax = df_distr[1:-3][['0_Label','1_Label']].plot(kind='bar', title ="Distribution of Average values across Target", figsize=(12, 8), legend=True, fontsize=12)
ax.set_xlabel("Numerical Features", fontsize=14)
ax.set_ylabel("Average Values", fontsize=14)
#ax.set_ylim(0,500000)
plt.show()

**Observations**

* Higher the length of URL ,or words in URL then more likely to be *Phishing* URL
* Clear distinctive pattern of *nb_links* field.Higher implies *Legitimate* URL
* *links_in_tags ,safe_anchor*  higher volume signals more towards *Legitmate* site


**Exploring Page Ranks feature w.r.t Target variable**

* Legitimate URLs form more of a gaussian distribution as the ranking increases , whereas the Phishing ones present a right skewed distribution.Presence of differentiating patterns here

In [ ]:
sns.catplot("page_rank", hue="target", data=df_data, kind="count", 
            palette={1:"green", 0:"blue"} ,height=5.0, aspect=11.7/8.27 )

**Note:**

* Aspects such has multicollinearity , variance captured by the features etc are applicable to Linear & Tree models as their performance tends to vary with these aspects.Since we are progressing with a Multi Layer Perceptron (MLP) , a neural net doesn't really care about all these transformations and raw data is what it needs to find the underlying patterns on it's own

#### Splitting dataset into Train & Test

In [ ]:
#Train & Test Set
X= df_data.iloc[: , 1:-1]
#y = upsampled_df['Churn']
y= df_data['target']

train_x,test_x,train_y,test_y = train_test_split(X,y,random_state=42)
print("\n--Training data samples--")
print(train_x.shape)


### Pytorch Neural Net Model
**Convert data into Pytorch Tensors**

In [ ]:
###First use a MinMaxscaler to scale all the features of Train & Test dataframes

scaler = preprocessing.MinMaxScaler()
x_train = scaler.fit_transform(train_x.values)
x_test =  scaler.fit_transform(test_x.values)

print("Scaled values of Train set \n")
print(x_train)
print("\nScaled values of Test set \n")
print(x_test)


###Then convert the Train and Test sets into Tensors

x_tensor =  torch.from_numpy(x_train).float()
y_tensor =  torch.from_numpy(train_y.values.ravel()).float()
xtest_tensor =  torch.from_numpy(x_test).float()
ytest_tensor =  torch.from_numpy(test_y.values.ravel()).float()

print("\nTrain set Tensors \n")
print(x_tensor)
print(y_tensor)
print("\nTest set Tensors \n")
print(xtest_tensor)
print(ytest_tensor)

### DataLoader to pass the data in batches to the model

In [ ]:
#Define a batch size , 
bs = 64
#Both x_train and y_train can be combined in a single TensorDataset, which will be easier to iterate over and slice
y_tensor = y_tensor.unsqueeze(1)
train_ds = TensorDataset(x_tensor, y_tensor)
#Pytorch’s DataLoader is responsible for managing batches. 
#You can create a DataLoader from any Dataset. DataLoader makes it easier to iterate over batches
train_dl = DataLoader(train_ds, batch_size=bs)


#For the validation/test dataset
ytest_tensor = ytest_tensor.unsqueeze(1)
test_ds = TensorDataset(xtest_tensor, ytest_tensor)
test_loader = DataLoader(test_ds, batch_size=32)

### MLP (Model)
Define the Layers , Activation function , Number of nodes for the MultiLayerPerceptron

#### Structure of MLP

* 2 Hidden Layers
* Normalizing the batch data usign batchnorm in between each layer
* Using ReLU Activation function between the layers
* Using dropout before sending to output
* Sigmoid at the output layer to make probabilities between 0 to 1

In [ ]:
n_input_dim = train_x.shape[1]

#Layer size
n_hidden1 = 300  # Number of hidden nodes
n_hidden2 = 100
n_output =  1   # Number of output nodes = for binary classifier


class ChurnModel(nn.Module):
    def __init__(self):
        super(ChurnModel, self).__init__()
        self.layer_1 = nn.Linear(n_input_dim, n_hidden1) 
        self.layer_2 = nn.Linear(n_hidden1, n_hidden2)
        self.layer_out = nn.Linear(n_hidden2, n_output) 
        
        
        self.relu = nn.ReLU()
        self.sigmoid =  nn.Sigmoid()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(n_hidden1)
        self.batchnorm2 = nn.BatchNorm1d(n_hidden2)
        
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.sigmoid(self.layer_out(x))
        
        return x
    

model = ChurnModel()
print(model)

#### Defining parameters (pretty much standard setting for Binary class problem)
* Loss computation function : Here using Binary Cross Entropy (BCE) which is defacto for Binary class problems
* Learning rate : Setting as 0.001 (can be optimized further)
* Optimizer : Using Adam and
* Epochs of Training : setting as 50

In [ ]:
#Loss Computation
loss_func = nn.BCELoss()
#Optimizer
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
epochs = 50

### Training the MLP Model
NN Steps flow

* Forward Propagation
* Loss computation
* Backpropagation
* Updating the parameters

In [ ]:
model.train()
train_loss = []
for epoch in range(epochs):
    #Within each epoch run the subsets of data = batch sizes.
    for xb, yb in train_dl:
        y_pred = model(xb)            # Forward Propagation
        loss = loss_func(y_pred, yb)  # Loss Computation
        optimizer.zero_grad()         # Clearing all previous gradients, setting to zero 
        loss.backward()               # Back Propagation
        optimizer.step()              # Updating the parameters 
    #print("Loss in iteration :"+str(epoch)+" is: "+str(loss.item()))
    train_loss.append(loss.item())
print('Last iteration loss value: '+str(loss.item()))

**Plotting the loss shows that model pretty much stabilized after 30 epochs itself**

In [ ]:
plt.plot(train_loss)
plt.show()

#### Test Dataset prediction on trained NN

In [ ]:
import itertools

y_pred_list = []
model.eval()
#Since we don't need model to back propagate the gradients in test set we use torch.no_grad()
# reduces memory usage and speeds up computation
with torch.no_grad():
    for xb_test,yb_test  in test_loader:
        y_test_pred = model(xb_test)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.detach().numpy())

#Takes arrays and makes them list of list for each batch        
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
#flattens the lists in sequence
ytest_pred = list(itertools.chain.from_iterable(y_pred_list))

#### Metrics that matter

* Precision
* Recall
* F1 Score
* Confusion Matrix

In [ ]:
y_true_test = test_y.values.ravel()
conf_matrix = confusion_matrix(y_true_test ,ytest_pred)
print("Confusion Matrix of the Test Set")
print("-----------")
print(conf_matrix)
print("Precision of the MLP :\t"+str(precision_score(y_true_test,ytest_pred)))
print("Recall of the MLP    :\t"+str(recall_score(y_true_test,ytest_pred)))
print("F1 Score of the Model :\t"+str(f1_score(y_true_test,ytest_pred)))

So here we used a Neural Net for a Tabular data classification problem and got pretty good performance.

**If you found this notebook helpful then please upvote , i'll be grateful** :-)